In [2]:
from peewee import *
mydb = PostgresqlDatabase("postgres",host="db",user="postgres",password="postgres")

class baseModel(Model):
    class Meta:
        database = mydb

In [3]:
class foodbank(baseModel):
    FB_ID = PrimaryKeyField()
    username = CharField(20, unique = True)
    name_of_org = CharField(100)
    primary_contact_name = CharField(100)
    phone_num = CharField(100)
    address = CharField(255, unique = True)
    email = CharField(100)
    date_joined = DateField() 
    city = CharField()
    state = CharField()
    zip = CharField()

class donor (baseModel):
    DO_ID = PrimaryKeyField()
    username = CharField(20, unique = True)
    name_of_org = CharField(100)
    type_of_org = CharField(100) 
    subscribed = BooleanField(default= False) 
    address = CharField(255, unique = True)
    primary_contact_name = CharField(100)
    phone_num = CharField(100)
    email = CharField(100)
    date_joined = DateField()
    city = CharField()
    state = CharField()
    zip = CharField()

class donation(baseModel):
    DN_ID = PrimaryKeyField()
    DO_ID = ForeignKeyField(donor, column_name='DO_ID')
    FB_ID = ForeignKeyField(foodbank, column_name='FB_ID')
    type_of_donation = CharField(255)
    quantity = IntegerField() # in pounds - check with Dr.Page if detailed breakdown of donations are needed, f so, we will create a different table to track quantity
    date_donated = DateField()

class fb_donation_request(baseModel):
    FB_REQ_ID = PrimaryKeyField()
    FB_ID = ForeignKeyField(foodbank,column_name='FB_ID')
    DO_ID = ForeignKeyField(donor,column_name='DO_ID')
    item = CharField(100)
    quantity = IntegerField()
    date_requested = DateField()
    status = CharField(255, default = 'pending') #

class do_donation_request(baseModel):
    DO_REQ_ID = PrimaryKeyField()
    DO_ID = ForeignKeyField(donor,column_name='DO_ID')
    FB_ID = ForeignKeyField(foodbank,column_name='FB_ID')
    item = CharField(100)
    quantity = IntegerField()
    date_requested = DateField()
    status = CharField(255, default = 'pending')

class documentation(baseModel):
    DOC_ID = PrimaryKeyField()
    DO_ID = ForeignKeyField(donor, null = True, column_name='DO_ID') 
    FB_ID = ForeignKeyField(foodbank,null = True, column_name='FB_ID') 
    type_of_documentation = CharField(255)
    date_obtained = DateField() 
    date_valid = DateField()

1. Submit documents. (HIGH) - CREATE

In [4]:
#This is created after it is run once
#Must change the DOC__ID in order for it to be a new insert and it will not give an error
#OR just comment this line out to see if the info exist
documentation.create(DOC_ID=33331, DO_ID=55214, FB_ID=84134, type_of_documentation='Registration', date_obtained = '2025-09-09', date_valid='2023-04-21')

#This is to make sure that the item is created
#It will be visible in the list of documentations after it has been successfully created above
query = (documentation
         .select()
         .join(foodbank, on=documentation.FB_ID == foodbank.FB_ID)
         .join(donor, on=documentation.DO_ID == donor.DO_ID))
for q in query:
    print ("DOC_ID: {} Food Bank Name:{} Donor Name:{} Type of Doc:{} Date Obtained:{} Date Valid:{}".format(q.DOC_ID, q.FB_ID.name_of_org, q.DO_ID.name_of_org, q.type_of_documentation, q.date_obtained, q.date_valid))

DOC_ID: 33331 Food Bank Name:Dictum Limited Donor Name:Turpis Ltd Type of Doc:Registration Date Obtained:2025-09-09 Date Valid:2023-04-21


2. View details of all document. (HIGH) - READ

In [5]:
#Displays the details of all documents in documentation
query = (documentation
         .select()
         .join(foodbank, on=documentation.FB_ID == foodbank.FB_ID)
         .join(donor, on=documentation.DO_ID == donor.DO_ID))
for q in query:
    print ("DOC_ID: {} Food Bank Name:{} Donor Name:{} Type of Doc:{} Date Obtained:{} Date Valid:{}".format(q.DOC_ID, q.FB_ID.name_of_org, q.DO_ID.name_of_org, q.type_of_documentation, q.date_obtained, q.date_valid))

DOC_ID: 33331 Food Bank Name:Dictum Limited Donor Name:Turpis Ltd Type of Doc:Registration Date Obtained:2025-09-09 Date Valid:2023-04-21


3. View donation received history. (HIGH) - READ

In [6]:
#View a list of all the donations that has been completed along with their details
query = (donation
         .select()
         .join(foodbank, on=donation.FB_ID == foodbank.FB_ID)
         .join(donor, on=donation.DO_ID == donor.DO_ID))
for q in query:
    print ("Donation ID: {} Donor Name:{} Food Bank Name:{} Type of Donation:{} Quantity:{} Date Donated:{}".format(q.DN_ID, q.DO_ID.name_of_org, q.FB_ID.name_of_org, q.type_of_donation, q.quantity, q.date_donated))

Donation ID: 37828 Donor Name:Metus Aliquam Limited Food Bank Name:Est Mollis Industries Type of Donation:pies Quantity:3 Date Donated:2023-12-23
Donation ID: 31474 Donor Name:Ac Mi PC Food Bank Name:Nunc PC Type of Donation:sandwiches Quantity:0 Date Donated:2023-07-03
Donation ID: 30178 Donor Name:Metus Aliquam Limited Food Bank Name:Eu Eleifend Corporation Type of Donation:pies Quantity:3 Date Donated:2023-10-04
Donation ID: 16415 Donor Name:Metus Aliquam Limited Food Bank Name:Arcu Incorporated Type of Donation:pasta Quantity:10 Date Donated:2024-08-26
Donation ID: 80595 Donor Name:Tellus Sem LLC Food Bank Name:Arcu Incorporated Type of Donation:sandwiches Quantity:5 Date Donated:2023-11-11
Donation ID: 27209 Donor Name:Mollis Nec LLC Food Bank Name:Molestie Tortor PC Type of Donation:soups Quantity:9 Date Donated:2023-10-12
Donation ID: 33177 Donor Name:Ipsum Porta Elit PC Food Bank Name:Molestie Tortor PC Type of Donation:noodles Quantity:6 Date Donated:2024-12-08
Donation ID: 87

4. View details of a donation. (HIGH) - READ

In [7]:
#Same as the above query, except this is for one specific donation
query = (donation
         .select()
         .join(foodbank, on=donation.FB_ID == foodbank.FB_ID)
         .join(donor, on=donation.DO_ID == donor.DO_ID)
         .where (donation.DN_ID == 37828))
for q in query:
    print ("Donation ID: {} Donor Name:{} Food Bank Name:{} Type of Donation:{} Quantity:{} Date Donated:{}".format(q.DN_ID, q.DO_ID.name_of_org, q.FB_ID.name_of_org, q.type_of_donation, q.quantity, q.date_donated))

Donation ID: 37828 Donor Name:Metus Aliquam Limited Food Bank Name:Est Mollis Industries Type of Donation:pies Quantity:3 Date Donated:2023-12-23


5. Sign up to the website. Insert usernmae, ID, etc. (HIGH) - CREATE

In [8]:
#User data will be created so that they can log in with an existing username
#Donor account creation
donor.create(DO_ID=52389, username='karina_lumaru', name_of_org='Karina_Lumaru', primary_contact_name='Karina Lumaru', phone_num='1-510-778-2257', address='190 Ferri St 1234', email='k_lumaru@gmail.com', city='Berea', state='KY', zip='40404', date_joined='2023-07-09', type_of_org='Individual', subscribed='Yes')

#This is to make sure that the item is created
#It will be visible in the list of donors after it has been successfully created above
query = (donor
         .select())
for q in query:
    print ("DO_ID: {} Donor Name:{} Donor Primary Contact Name:{} Type Of Org:{} Subscribed :{}".format(q.DO_ID, q.name_of_org, q.primary_contact_name, q.type_of_org, q.subscribed))

DO_ID: 55214 Donor Name:Turpis Ltd Donor Primary Contact Name:Jesse Lopez Type Of Org:Individual Subscribed :True
DO_ID: 36580 Donor Name:Mollis Nec LLC Donor Primary Contact Name:Angelica Barnes Type Of Org:Individual Subscribed :True
DO_ID: 93180 Donor Name:Gravida PC Donor Primary Contact Name:Naida Cash Type Of Org:Individual Subscribed :True
DO_ID: 44241 Donor Name:Magna Suspendisse Associates Donor Primary Contact Name:Boris Hines Type Of Org:Organisation Subscribed :True
DO_ID: 79989 Donor Name:Dictum Placerat PC Donor Primary Contact Name:Hedy Frye Type Of Org:Individual Subscribed :True
DO_ID: 60159 Donor Name:Tellus Sem LLC Donor Primary Contact Name:Hiram Reid Type Of Org:Organisation Subscribed :True
DO_ID: 63353 Donor Name:Ipsum Porta Elit PC Donor Primary Contact Name:Elton Melendez Type Of Org:Organisation Subscribed :True
DO_ID: 85314 Donor Name:Ac Mi PC Donor Primary Contact Name:Dai Merrill Type Of Org:Individual Subscribed :True
DO_ID: 52309 Donor Name:Hendrerit Cons

In [9]:
#User data will be created so that they can log in with an existing username
#Food bank account creation
#It can only be created once so it gives error after it run once
# COmment this part out after running once to see it in the results of food banks
foodbank.create(FB_ID=84890, username='fb_user84890', name_of_org='Creative Limited', primary_contact_name='Anna Luis', phone_num='(805) 855-8902', address='1721 Oliver Av.', email='lame.email@google.org', city='Berea', state='KY', zip='40404', date_joined='2024-03-23')

#This is to make sure that the item is created
#It will be visible in the list of food banks after it has been successfully created above
query = (foodbank
         .select())
for q in query:
    print ("FB_ID: {} Username:{} Name Of Org: {} Donor Primary Contact Name:{} Phone Num: {} Address: {} Email: {} City: {} State: {} Zip Code: {} Date Joined:{}".format(q.FB_ID, q.username, q.name_of_org, q.primary_contact_name,q.phone_num, q.address,q.email, q.city, q.state, q.zip, q.date_joined))

FB_ID: 84134 Username:fb_user84134 Name Of Org: Dictum Limited Donor Primary Contact Name:Gavin Olsen Phone Num: (805) 811-5218 Address: 1721 Nascetur Av. Email: curabitur.sed@google.edu City: New York State: NY Zip Code: 10001 Date Joined:2024-04-13
FB_ID: 44204 Username:fb_user44204 Name Of Org: Nunc PC Donor Primary Contact Name:Omar Compton Phone Num: (495) 487-2527 Address: P.O. Box 394, 9121 Adipiscing Rd. Email: tristique.senectus@outlook.couk City: Los Angeles State: CA Zip Code: 90001 Date Joined:2025-03-23
FB_ID: 14789 Username:fb_user14789 Name Of Org: Libero Est LLP Donor Primary Contact Name:Ciara Hopper Phone Num: (305) 451-8544 Address: 8525 Praesent Road Email: rutrum@icloud.com City: Chicago State: IL Zip Code: 60601 Date Joined:2023-10-31
FB_ID: 90226 Username:fb_user90226 Name Of Org: Arcu Incorporated Donor Primary Contact Name:Savannah Brennan Phone Num: 1-975-967-4234 Address: 891-8625 Eleifend Street Email: sed.libero.proin@aol.edu City: Houston State: TX Zip Cod

6. View details of all donors (phone number/ email/name/ date joined. (HIGH) - READ

In [10]:
#displays phone number, primary contact person, address, email, and joined date of all donors 
query = (donor
         .select())
for q in query:
    print ("Donor Phone Number: {} Donor Primary Contact Person:{} Donor Address:{} Donor Email:{} Date Joined:{}".format(q.phone_num, q.primary_contact_name, q.address, q.email, q.date_joined))

Donor Phone Number: 1-510-766-9562 Donor Primary Contact Person:Jesse Lopez Donor Address:P.O. Box 966 174 Nec St. Donor Email:eu.turpis@hotmail.net Date Joined:2023-05-02
Donor Phone Number: 1-644-561-3419 Donor Primary Contact Person:Angelica Barnes Donor Address:Ap #897-4039 Diam. Rd. Donor Email:donec.sollicitudin@outlook.ca Date Joined:2024-02-24
Donor Phone Number: (284) 430-8637 Donor Primary Contact Person:Naida Cash Donor Address:176-7009 Et, Road Donor Email:elit.erat@yahoo.co.uk Date Joined:2023-04-26
Donor Phone Number: 1-684-837-5382 Donor Primary Contact Person:Boris Hines Donor Address:Ap #680-6297 Ac Av. Donor Email:donec.tincidunt@google.net Date Joined:2023-05-19
Donor Phone Number: 1-251-352-9093 Donor Primary Contact Person:Hedy Frye Donor Address:219-3578 Erat. Ave Donor Email:donec.elementum@outlook.com Date Joined:2024-05-26
Donor Phone Number: (232) 837-8646 Donor Primary Contact Person:Hiram Reid Donor Address:P.O. Box 370, 9694 Purus, Ave Donor Email:velit.ege

7. Search for donors in the area (city/zip code/state). (HIGH) - READ

In [11]:
#Filter by city
query = (donor
         .select()
         .where (donor.city=='Los Angeles'))
for q in query:
    print ("Donor Phone Number: {} Donor Primary Contact Person:{} Donor Address:{} Donor City: {} Donor Email:{} Date Joined:{}".format(q.phone_num, q.primary_contact_name, q.address, q.city, q.email, q.date_joined))


Donor Phone Number: 1-644-561-3419 Donor Primary Contact Person:Angelica Barnes Donor Address:Ap #897-4039 Diam. Rd. Donor City: Los Angeles Donor Email:donec.sollicitudin@outlook.ca Date Joined:2024-02-24


In [12]:
#Filter by state
query = (donor
         .select()
         .where (donor.state=='TX'))
for q in query:
    print ("Donor Phone Number: {} Donor Primary Contact Person:{} Donor Address:{} Donor State: {} Donor Email:{} Date Joined:{}".format(q.phone_num, q.primary_contact_name, q.address, q.state, q.email, q.date_joined))

Donor Phone Number: 1-684-837-5382 Donor Primary Contact Person:Boris Hines Donor Address:Ap #680-6297 Ac Av. Donor State: TX Donor Email:donec.tincidunt@google.net Date Joined:2023-05-19
Donor Phone Number: 1-187-127-8675 Donor Primary Contact Person:Elton Melendez Donor Address:P.O. Box 477, 1799 Hendrerit Rd. Donor State: TX Donor Email:gravida@aol.co.uk Date Joined:2024-06-19
Donor Phone Number: (466) 332-1228 Donor Primary Contact Person:Aspen Weaver Donor Address:859-2853 Nec, Rd. Donor State: TX Donor Email:cras.dolor@protonmail.net Date Joined:2023-06-13


In [13]:
#Filter by zip code
query = (donor
         .select()
         .where (donor.zip=='40404'))
for q in query:
    print ("Donor Phone Number: {} Donor Primary Contact Person:{} Donor Address:{} Donor Zip Code: {} Donor Email:{} Date Joined:{}".format(q.phone_num, q.primary_contact_name, q.address, q.zip, q.email, q.date_joined))

Donor Phone Number: 1-510-778-2257 Donor Primary Contact Person:Karina Lumaru Donor Address:190 Ferri St 1234 Donor Zip Code: 40404 Donor Email:k_lumaru@gmail.com Date Joined:2023-07-09


8. Request donations by filling out a form (for donor). (HIGH) - CREATE.

In [14]:
#This has already been run so it is already created
#Must change the FB_REQ_ID in order for it to be a new insert and it will not give an error
#OR just comment this out after running once to see it in the list of fb_donation_requests without error
fb_donation_request.create(FB_REQ_ID=13369, DO_ID=52309, FB_ID=18607, item="pineapples", quantity = 9, status="pending", date_requested="2024-04-05")

#This is to make sure that the item is create
#It will be visible in the list of fb_donation_request after it has been successfully created above
query = (fb_donation_request
         .select()
         .join(foodbank, on=fb_donation_request.FB_ID == foodbank.FB_ID)
         .join(donor, on=fb_donation_request.DO_ID == donor.DO_ID))
for q in query:
    print ("Food Bank Name:{} Donor Name:{} Item:{} Quantity:{} Date Requested:{}".format(q.FB_ID.name_of_org, q.DO_ID.name_of_org, q.item, q.quantity, q.date_requested))

Food Bank Name:Dictum Limited Donor Name:Mollis Nec LLC Item:shoes Quantity:3 Date Requested:2021-12-01
Food Bank Name:Libero Est LLP Donor Name:Mollis Nec LLC Item:apples Quantity:50 Date Requested:2022-11-01
Food Bank Name:Est Mollis Industries Donor Name:Hendrerit Consectetuer Cursus PC Item:pineapples Quantity:9 Date Requested:2024-04-05


9. View donation requests made by donors. (HIGH) - READ

In [15]:
#displays list of all donation requests made by donors
query = (do_donation_request
         .select()
         .join(foodbank, on=do_donation_request.FB_ID == foodbank.FB_ID)
         .join(donor, on=do_donation_request.DO_ID == donor.DO_ID))
for q in query:
    print ("Food Bank Name:{} Donor Name:{} Item:{} Quantity:{} Date Requested:{}".format(q.FB_ID.name_of_org, q.DO_ID.name_of_org, q.item, q.quantity, q.date_requested))

Food Bank Name:Nunc PC Donor Name:Turpis Ltd Item:hats Quantity:5 Date Requested:2023-11-26
Food Bank Name:Est Mollis Industries Donor Name:Turpis Ltd Item:oranges Quantity:9 Date Requested:2024-02-01


10. View details of a donation requests from a specific donor. (HIGH) - READ

In [16]:
#same as the above query, except this is requets made by a specific donor
query = (do_donation_request
         .select()
         .join(foodbank, on=do_donation_request.FB_ID == foodbank.FB_ID)
         .join(donor, on=do_donation_request.DO_ID == donor.DO_ID)
         .where (donor.name_of_org== 'Turpis Ltd'))
for q in query:
    print ("Food Bank Name:{} Donor Name:{} Item:{} Quantity:{} Date Requested:{}".format(q.FB_ID.name_of_org, q.DO_ID.name_of_org, q.item, q.quantity, q.date_requested))

Food Bank Name:Nunc PC Donor Name:Turpis Ltd Item:hats Quantity:5 Date Requested:2023-11-26
Food Bank Name:Est Mollis Industries Donor Name:Turpis Ltd Item:oranges Quantity:9 Date Requested:2024-02-01


11. View personal information of a specific donor. (HIGH) - READ

In [17]:
query = (donor
         .select(donor.name_of_org, donor.type_of_org, donor.primary_contact_name, donor.subscribed, donor.city, donor.state, donor.zip)
         .where(donor.name_of_org == 'Mollis Nec LLC'))

# Loop through the results and print them
for q in query:
    print("Name of Organization:{} Type of Organization:{} Primary Contact Name:{} Subscribed:{} City:{} State:{} Zip:{}".format(q.name_of_org, q.type_of_org, q.primary_contact_name, q.subscribed, q.city, q.state, q.zip))
    


Name of Organization:Mollis Nec LLC Type of Organization:Individual Primary Contact Name:Angelica Barnes Subscribed:True City:Los Angeles State:CA Zip:90001


12. Edit personal information. (HIGH) - UPDATE

In [18]:
# Execute the update query
query = donor.update(
    address='2576 Blackberry Ridge Blvd',
    phone_num='865-990-8786',
    primary_contact_name='Pedro Luis',
    email='pedro@gmail,com',
    name_of_org='The Life of Pedro',
    type_of_org='Individual',
    city='Houston',
    state='TX',
    zip='90001'
).where(donor.DO_ID == '36580')

# Execute the query
query.execute()

#to check if the update occurred
query = (donor.select()
         .where(donor.DO_ID==36580))
for q in query:
    print("Name of Organization:{} Type of Organization:{} Primary Contact Name:{} Subscribed:{} City:{} State:{} Zip:{}".format(q.name_of_org, q.type_of_org, q.primary_contact_name, q.subscribed, q.city, q.state, q.zip))

Name of Organization:The Life of Pedro Type of Organization:Individual Primary Contact Name:Pedro Luis Subscribed:True City:Houston State:TX Zip:90001


13. Search for details of all documents. (HIGH) - READ

In [19]:
query = (documentation
         .select(documentation.DOC_ID, documentation.FB_ID, documentation.DO_ID, documentation.type_of_documentation, documentation.date_obtained, documentation.date_valid))

# Loop through the results and print them
for q in query:
    print("DOC_ID:{} FB_ID:{} DO_ID:{} Type of Documentation:{} Date Obtained:{} Date Valid:{}".format(q.DOC_ID, q.FB_ID, q.DO_ID, q.type_of_documentation, q.date_obtained, q.date_valid))


DOC_ID:32128 FB_ID:None DO_ID:55214 Type of Documentation:Finances Date Obtained:2029-05-08 Date Valid:2023-06-23
DOC_ID:69644 FB_ID:44204 DO_ID:None Type of Documentation:Licenses Date Obtained:2027-06-21 Date Valid:2024-07-08
DOC_ID:25464 FB_ID:None DO_ID:79989 Type of Documentation:Quality Assurance Date Obtained:2033-07-13 Date Valid:2024-09-18
DOC_ID:70638 FB_ID:31557 DO_ID:None Type of Documentation:Licenses Date Obtained:2026-04-30 Date Valid:2023-11-24
DOC_ID:56902 FB_ID:None DO_ID:93180 Type of Documentation:Quality Assurance Date Obtained:2027-04-14 Date Valid:2023-06-17
DOC_ID:13906 FB_ID:60551 DO_ID:None Type of Documentation:Licenses Date Obtained:2033-06-19 Date Valid:2024-02-20
DOC_ID:65680 FB_ID:None DO_ID:85314 Type of Documentation:Finances Date Obtained:2025-10-11 Date Valid:2023-09-14
DOC_ID:11174 FB_ID:18607 DO_ID:None Type of Documentation:Licenses Date Obtained:2029-01-08 Date Valid:2024-08-14
DOC_ID:66381 FB_ID:None DO_ID:52309 Type of Documentation:Quality Assu

14. Search for food banks in the area (filter by state/ city/ zip code). (HIGH) - READ

In [20]:
#Filter by city
query = (foodbank
         .select()
         .where (foodbank.city=='Los Angeles'))
for q in query:
    print (" Food Bank Phone Number: {} Food Bank Primary Contact Person:{} Food Bank Address:{} Food Bank City: {} Food Bank State: {} Food Bank Zip Code: {} Food Bank Email:{} Date Joined:{}".format(q.phone_num, q.primary_contact_name, q.address,q.city, q.state, q.zip, q.email, q.date_joined))

 Food Bank Phone Number: (495) 487-2527 Food Bank Primary Contact Person:Omar Compton Food Bank Address:P.O. Box 394, 9121 Adipiscing Rd. Food Bank City: Los Angeles Food Bank State: CA Food Bank Zip Code: 90001 Food Bank Email:tristique.senectus@outlook.couk Date Joined:2025-03-23


In [21]:
#Filter by state
query = (foodbank
         .select()
         .where (foodbank.state=='TX'))
for q in query:
    print (" Food Bank Phone Number: {} Food Bank Primary Contact Person:{} Food Bank Address:{} Food Bank City: {} Food Bank State: {} Food Bank Zip Code: {} Food Bank Email:{} Date Joined:{}".format(q.phone_num, q.primary_contact_name, q.address,q.city, q.state, q.zip, q.email, q.date_joined))

 Food Bank Phone Number: 1-975-967-4234 Food Bank Primary Contact Person:Savannah Brennan Food Bank Address:891-8625 Eleifend Street Food Bank City: Houston Food Bank State: TX Food Bank Zip Code: 77001 Food Bank Email:sed.libero.proin@aol.edu Date Joined:2023-11-30
 Food Bank Phone Number: (777) 451-0807 Food Bank Primary Contact Person:Sonia Petty Food Bank Address:685-5050 Etiam Ave Food Bank City: San Antonio Food Bank State: TX Food Bank Zip Code: 78201 Food Bank Email:nec.metus@google.couk Date Joined:2023-08-18
 Food Bank Phone Number: 1-446-684-2218 Food Bank Primary Contact Person:Valentine Figueroa Food Bank Address:P.O. Box 650, 4463 Feugiat Road Food Bank City: Dallas Food Bank State: TX Food Bank Zip Code: 75201 Food Bank Email:ornare.facilisis@protonmail.edu Date Joined:2023-09-25


In [22]:
#Filter by zip code
query = (foodbank
         .select()
         .where (foodbank.zip=='60601'))
for q in query:
    print (" Food Bank Phone Number: {} Food Bank Primary Contact Person:{} Food Bank Address:{} Food Bank City: {} Food Bank State: {} Food Bank Zip Code: {} Food Bank Email:{} Date Joined:{}".format(q.phone_num, q.primary_contact_name, q.address,q.city, q.state, q.zip, q.email, q.date_joined))

 Food Bank Phone Number: (305) 451-8544 Food Bank Primary Contact Person:Ciara Hopper Food Bank Address:8525 Praesent Road Food Bank City: Chicago Food Bank State: IL Food Bank Zip Code: 60601 Food Bank Email:rutrum@icloud.com Date Joined:2023-10-31


15. View details of all food banks. (HIGH) - READ

In [23]:
query = (foodbank.select(foodbank.FB_ID, foodbank.username, foodbank.name_of_org, foodbank.address, foodbank.phone_num, foodbank.primary_contact_name, foodbank.email, foodbank.date_joined))

# Loop through the results and print them
for q in query:
    print("FB_ID:{} Username:{} Name of Organization:{} Address:{} Phone Number:{} Primary Contact Name:{} Email:{} Date Joined:{}".format(q.FB_ID, q.username, q.name_of_org, q.address, q.phone_num, q.primary_contact_name, q.email, q.date_joined))


FB_ID:84134 Username:fb_user84134 Name of Organization:Dictum Limited Address:1721 Nascetur Av. Phone Number:(805) 811-5218 Primary Contact Name:Gavin Olsen Email:curabitur.sed@google.edu Date Joined:2024-04-13
FB_ID:44204 Username:fb_user44204 Name of Organization:Nunc PC Address:P.O. Box 394, 9121 Adipiscing Rd. Phone Number:(495) 487-2527 Primary Contact Name:Omar Compton Email:tristique.senectus@outlook.couk Date Joined:2025-03-23
FB_ID:14789 Username:fb_user14789 Name of Organization:Libero Est LLP Address:8525 Praesent Road Phone Number:(305) 451-8544 Primary Contact Name:Ciara Hopper Email:rutrum@icloud.com Date Joined:2023-10-31
FB_ID:90226 Username:fb_user90226 Name of Organization:Arcu Incorporated Address:891-8625 Eleifend Street Phone Number:1-975-967-4234 Primary Contact Name:Savannah Brennan Email:sed.libero.proin@aol.edu Date Joined:2023-11-30
FB_ID:95899 Username:fb_user95899 Name of Organization:Lacus Ut Ltd Address:851-4808 Viverra. Avenue Phone Number:(628) 573-3280 

16. Request to donate by filling out a form. (HIGH) - CREATE

In [24]:
#This has already been run so it is already created
#Must change the DO_REQ_ID in order for it to be a new insert and it will not give an error
#OR just comment this part out after running once so it does not give error and result can be seen on the list of do_donation_requests
do_donation_request.create(DO_REQ_ID=13335, DO_ID=52309, FB_ID=18607, item="melons", quantity = 9, status="pending", date_requested="2024-04-05")

#This is to make sure that the item is create
#It will be visible in the list of do_donation_requests after it has been successfully created above
query = (do_donation_request
         .select()
         .join(foodbank, on=do_donation_request.FB_ID == foodbank.FB_ID)
         .join(donor, on=do_donation_request.DO_ID == donor.DO_ID))
for q in query:
    print ("Food Bank Name:{} Donor Name:{} Item:{} Quantity:{} Date Requested:{}".format(q.FB_ID.name_of_org, q.DO_ID.name_of_org, q.item, q.quantity, q.date_requested))


Food Bank Name:Nunc PC Donor Name:Turpis Ltd Item:hats Quantity:5 Date Requested:2023-11-26
Food Bank Name:Est Mollis Industries Donor Name:Turpis Ltd Item:oranges Quantity:9 Date Requested:2024-02-01
Food Bank Name:Est Mollis Industries Donor Name:Hendrerit Consectetuer Cursus PC Item:melons Quantity:9 Date Requested:2024-04-05


17. Accept/decline to make a donation (foodbanks can ask donors who have donated before to donate again). (HIGH) - UPDATE

In [25]:
#If a request was accepted by a donor
row=fb_donation_request.get(fb_donation_request.FB_REQ_ID == 00000)
print ("FB_REQ_ID: {} Food Bank ID: {} Donor ID: {} Date Requested: {} Status: {}".format(row.FB_REQ_ID, row.FB_ID, row.DO_ID, row.date_requested, row.status))
row.status = "accepted"
row.save()


FB_REQ_ID: 0 Food Bank ID: 84134 Donor ID: 36580 Date Requested: 2021-12-01 Status: pending


1

In [26]:
#If a request was declined by a donor
row=fb_donation_request.get(fb_donation_request.FB_REQ_ID == 00000)
print ("FB_REQ_ID: {} Food Bank ID: {} Donor ID: {} Date Requested: {} Status: {}".format(row.FB_REQ_ID, row.FB_ID, row.DO_ID, row.date_requested, row.status))
row.status = "declined"
row.save()

FB_REQ_ID: 0 Food Bank ID: 84134 Donor ID: 36580 Date Requested: 2021-12-01 Status: accepted


1

18. View donation requests from food banks. (HIGH) - READ

In [27]:
#This shows the records in in the fb_donation_request, which are all requests made by food banks to a specfic past donor
#It displays the name of the food bank, the donor they requested donations from, the item, quantity, and when they requested
query = (fb_donation_request
         .select()
         .join(foodbank, on=fb_donation_request.FB_ID == foodbank.FB_ID)
         .join(donor, on=fb_donation_request.DO_ID == donor.DO_ID))
for q in query:
    print ("Food Bank Name:{} Donor Name:{} Item:{} Quantity:{} Date Requested:{}".format(q.FB_ID.name_of_org, q.DO_ID.name_of_org, q.item, q.quantity, q.date_requested))

Food Bank Name:Libero Est LLP Donor Name:The Life of Pedro Item:apples Quantity:50 Date Requested:2022-11-01
Food Bank Name:Est Mollis Industries Donor Name:Hendrerit Consectetuer Cursus PC Item:pineapples Quantity:9 Date Requested:2024-04-05
Food Bank Name:Dictum Limited Donor Name:The Life of Pedro Item:shoes Quantity:3 Date Requested:2021-12-01


19. View details of a donation requested by a food bank. (HIGH) - READ

In [28]:
#This shows the details of a specifc donation request that was made by a food bank to a donor
#This is similar to the previous one, except it has a where condition as it is seraching for donation request from a specifc food bank
query = (fb_donation_request
         .select()
         .join(foodbank, on=fb_donation_request.FB_ID == foodbank.FB_ID)
         .join(donor, on=fb_donation_request.DO_ID == donor.DO_ID)
         .where (foodbank.name_of_org == 'Dictum Limited'))
for q in query:
    print ("Food Bank Name:{} Donor Name:{} Item:{} Quantity:{} Date Requested:{}".format(q.FB_ID.name_of_org, q.DO_ID.name_of_org, q.item, q.quantity, q.date_requested))

Food Bank Name:Dictum Limited Donor Name:The Life of Pedro Item:shoes Quantity:3 Date Requested:2021-12-01


5. View personal information of all donors. (HIGH) - READ

In [29]:
#donor information
rows=donor.select()
for row in rows:
   print ("Name of Org: {} Type of Org: {} Primary Contact Name: {} Subscribed: {} City: {} State: {} Zip: {}".format(row.name_of_org, row.type_of_org, row.primary_contact_name, row.subscribed, row.city, row.state, row.zip))


In [35]:
query = (documentation
         .select()
         .join(donor, on=documentation.DO_ID==donor.DO_ID)
         .where(donor.DO_ID==55214))
for q in query:
    print ("Donor Name:{} DOC name:{}".format(q.DO_ID.name_of_org, q.type_of_documentation))


Donor Name:Turpis Ltd DOC name:Finances
Donor Name:Turpis Ltd DOC name:Quality Assurance
Donor Name:Turpis Ltd DOC name:Registration
